### Here, I tried it with the model v4

In [1]:
# importing the Python libraries module
import gymnasium as gym
import ale_py
from  IPython.display import clear_output

from stable_baselines3 import PPO, DQN, A2C
from stable_baselines3.common.env_util import make_atari_env

from stable_baselines3.common.vec_env import VecFrameStack, VecVideoRecorder

import matplotlib.pyplot as plt
import os
import re

# INSTALL MOVIEPY => pip install gymnasium[other]
env_name = 'AssaultNoFrameskip-v4'

# create multiple connected envs at same time by using make_atari_env + SubprocVecEnv
env = make_atari_env(env_name, wrapper_kwargs={'terminal_on_life_loss': False, 'clip_reward': False})

env = VecFrameStack(env, n_stack=4)

# wrap the environment to capture video
# ADJUST THE VIDEO LENGTH IF YOU DON'T GET ANY VIDEOS PRODUCED (1000 is often good
# try also 500 or 2000)
env = VecVideoRecorder(
    env, 
    video_folder="./videos",
    record_video_trigger=lambda x: True,
    video_length=1000
)

#### The error: IndexError: list index out of range, is caused by due to this line: return (numbers[0], numbers[1]). And due to when the filename does not contain at least two numbers, so trying to access numbers[1] fails.
#### So every file in the logs/ folder — even ones that might not be model files such as .json, .txt, .mp4, or maybe files with wrong names like log_config.txt or checkpoint.zip without numbers. 
#### In this instance, re.findall(r'\d+', filename) not returning at least two numbers, accessing numbers[1] fails. The reason 



In [3]:
# get all checkpoint files in the folder
models = os.listdir("logs/")

# ChatGPT generated originally (AND SURE LOOKS LIKE IT!!!!)'
# function to extract numbers from the checkpoint file name
def extract_numbers(filename):
    numbers = list(map(int, re.findall(r'\d+', filename)))
    return (numbers[0], numbers[1])

# custom sorting
sorted_files = sorted(models, key=extract_numbers)
models = sorted_files

# use latest model in the logs-folder
model_path = models[-1]
model = PPO.load(f"logs/{model_path}", env=env)

print("Using the latest model:")
print(model_path)

IndexError: list index out of range

In [ ]:
obs = env.reset()
total_reward = 0
done = False

while not done:
    action, _state = model.predict(obs)
    obs, reward, done, info = env.step(action)

    image = env.render("rgb_array")
    clear_output(wait=True)
    plt.imshow(image)
    plt.axis("off")
    plt.show()
    total_reward += reward

print("Total reward:", total_reward)
env.close()